# California housing prices 
.
Versão com o PySpark do kernel [California housing prices](https://github.com/robson-rsp/datascience/blob/main/regression/california_housing_prices.ipynb). Este notebook foi escrito no ambiente do Databricks e exportado como arquivo IPython .ipynb. Os textos daqui são cópias dos originais do link.
---
A tarefa neste kernel é criar um modelo de Machine Learning que estime preços de imóveis no estado da Califórnia baseado em características, como:

*   latitude/longitude: Localização do imóvel
*   housing_median_age: Idade média dos imóveis naquela quadra
*   totalRooms: Número total de cômodos dentro de um quarteirão
*   totalBedrooms: Número total de quartos dentro de um quarteirão
*   population: Número total de pessoas que moram em uma quadra
*   households: Número total de habitações em uma quadra
*   medianIncome: Renda média das famílias de uma quadra (medida em dezenas de milhares de dólares americanos)
*   medianHouseValue: Valor médio dos imóveis de uma quadra (medido em dólares americanos)
*   oceanProximity: Localização do imóvel em relação ao oceano/mar

Obs. A média populacional desses quarteirões é de 600 até 3000 pessoas.

Fonte: [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices)

In [0]:

housing = spark.read.format("csv") \
          .option("inferSchema", "True") \
          .option("header", "True") \
          .option("sep", ",") \
          .load("/FileStore/tables/housing.csv")

# EDA

In [0]:
housing.sample(fraction=0.01).toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.26,37.85,52.0,2643.0,626.0,1212.0,620.0,1.9167,159200.0,NEAR BAY
1,-122.30,37.89,46.0,1520.0,402.0,815.0,375.0,2.8036,211600.0,NEAR BAY
2,-122.06,37.54,20.0,6483.0,1068.0,3526.0,1060.0,5.0838,248200.0,NEAR BAY
3,-121.87,37.67,10.0,4337.0,800.0,1813.0,743.0,5.5000,247200.0,INLAND
4,-121.80,39.75,11.0,7212.0,1355.0,3264.0,1264.0,3.1125,122600.0,INLAND
...,...,...,...,...,...,...,...,...,...,...
184,-119.06,34.22,13.0,4175.0,1321.0,2257.0,1271.0,3.1446,177100.0,<1H OCEAN
185,-119.09,34.22,8.0,40.0,10.0,309.0,16.0,4.0208,52500.0,NEAR OCEAN
186,-118.83,34.14,16.0,1316.0,194.0,450.0,173.0,10.1597,500001.0,NEAR OCEAN
187,-118.86,34.21,26.0,3354.0,659.0,2020.0,648.0,4.1576,211800.0,<1H OCEAN


In [0]:
print(f'shape={housing.count(), len(housing.columns)}')

shape=(20640, 10)


In [0]:
housing.summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max").toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,25%,-121.8,33.93,18.0,1447.0,296.0,787.0,280.0,2.5625,119600.0,None
5,50%,-118.49,34.26,29.0,2127.0,435.0,1166.0,409.0,3.5347,179700.0,None
6,75%,-118.01,37.71,37.0,3146.0,647.0,1724.0,605.0,4.7426,264700.0,None
7,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Verificando a ocorrência de campos NaN.

In [0]:
for name in housing.columns:
    mask = housing[name].isNull()
    print(f'{name}: {housing.where(mask).count()}')

longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 207
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0


# Data visualization

Os gráficos foram criados utilizando as ferramentas de visualização nativas do Databricks. O resultado não pode ser visto em um arquivo .ipynb

# Train/Test split

In [0]:
train_set, test_set = housing.randomSplit([0.8, 0.2])

# Feature engineering

In [0]:
from pyspark.ml            import Pipeline
from pyspark.ml.feature    import Imputer, StandardScaler, StringIndexer, VectorAssembler
from pyspark.sql.functions import *

Imports necessários para criar uma classe custom transformer.

In [0]:
from pyspark                 import keyword_only
from pyspark.ml              import Transformer
from pyspark.ml.param.shared import HasInputCols, HasOutputCol, Params
from pyspark.sql.functions   import udf

## Categorical features

**Etapa ##:** Atributos nominais: Codificar

In [0]:
stringindexer = StringIndexer(inputCol='ocean_proximity', outputCol='ocean_proximity_')
#stringindexer_model = stringindexer.fit(train_set)
#train_set = stringindexer_model.transform(train_set)
#test_set  = stringindexer.transform(test_set)

## Numerical features

**Etapa ##:** NaN values: Imputar

In [0]:
imputer = Imputer(strategy='mean', inputCol='total_bedrooms', outputCol='total_bedrooms_')
#imputer_model = imputer.fit(train_set)
#train_set = imputer_model.transform(train_set)
#test_set  = imputer_model.transform(test_set)

**Etapa ##:** Os atributos 'total_rooms' e 'total_bedrooms' não fazem sentido se eu deixá-los da forma como estão. 'total_rooms' refere-se à quantidade total de cômodos que há em cada quadra. 'total_bedrooms' refere-se à quantidade total de quartos que há em cada quadra. Vou criar novos atributos a partir dos existentes. Vou fazer isso dentro de uma classe para usar a classe Pipeline.
1. **mean_bedrooms_households:** Quantidade média de quartos por habitação.
1. **mean_rooms_households:** Quantidade média de cômodos por habitação.
1. **mean_population_households:** Quantidade média de moradores por imóvel.

In [0]:
class MeanTransformer(Transformer, HasInputCols, HasOutputCol):
    @keyword_only
    def __init__(self, inputCols=None, outputCol=None):
        super().__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)
    
    @keyword_only
    def setParams(self, inputCols=None, outputCol=None):
        kwargs = self._input_kwargs
        self._set(**kwargs)

    def setInputCols(self, inputCols=None):
        self.setParams(inputCols=inputCols)

    def setOutputCol(self, outputCol):
        self.setParams(outputCol=outputCol)

    def _transform(self, dataframe):
        cols_in  = self.getInputCols()
        col_out = self.getOutputCol()
        dataframe = dataframe.withColumn(col_out, dataframe[cols_in[0]] / dataframe[cols_in[1]])
        return dataframe



meantransformer_bedrooms   = MeanTransformer(inputCols=['total_bedrooms_', 'households'], outputCol='mean_bedrooms_households')
meantransformer_rooms      = MeanTransformer(inputCols=['total_rooms', 'households'], outputCol='mean_rooms_households')
meantransformer_population = MeanTransformer(inputCols=['population', 'households'], outputCol='mean_population_households')

**Etapa ##:** Unir todas as colunas em uma única que será utilizada pelos modelos de Machine Learning do PySpark. Essa etapa dever ser feita antes da padronização dos atributos com StandardScaler.

In [0]:
cols = ['longitude', 'latitude', 'housing_median_age', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity_', 'total_bedrooms_', 'mean_bedrooms_households', 'mean_rooms_households', 'mean_population_households']

vectorassembler = VectorAssembler(inputCols=cols, outputCol='X')
#train_set = vectorassembler.transform(train_set)
#test_set  = vectorassembler.transform(test_set)

**Etapa ##:** Padronização dos atributos com StandardScaler.

In [0]:
standardscaler = StandardScaler(inputCol='X', outputCol='X_')
#standardscaler_model = standardscaler.fit(train_set)
#train_set = standardscaler_model.transform(train_set)
#test_set  = standardscaler_model.transform(test_set)

**Etapa final: Criação do pipeline de transformações.**

In [0]:
s = [stringindexer, imputer, meantransformer_bedrooms, meantransformer_rooms, meantransformer_population, vectorassembler, standardscaler]
pipe = Pipeline(stages=s)

pipe_model = pipe.fit(train_set)
train_set = pipe_model.transform(train_set)
test_set  = pipe_model.transform(test_set)

Selecionar X e y dos conjuntos de treino e teste. Vou renomear "X_" para "X"  e "median_house_value" para "y" assim os conjuntos ficarão de acordo com o padrão de nomenclatura.

In [0]:
train_set = train_set.select('X_', 'median_house_value')
test_set  = test_set.select('X_', 'median_house_value')

train_set = train_set.withColumnRenamed(existing='X_', new='X')
test_set  = test_set.withColumnRenamed(existing='X_', new='X')

train_set = train_set.withColumnRenamed(existing='median_house_value', new='y')
test_set  = test_set.withColumnRenamed(existing='median_house_value', new='y')

# Model training


In [0]:
from pyspark.ml.regression import GBTRegressor, LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning     import CrossValidator, ParamGridBuilder

## Baseline model
Como o PySpark não possui uma classe como DummyRegressor do Scikit Learn, vou criar o resultado que ela teria caso existisse. O padrão dela é prever sempre a média aritmética da label do conjunto de treinamento como sua previsão. Portanto é isso que farei, mas sem o uso de classes.

In [0]:
m = train_set.select(mean('y')).collect()[0][0] # média aritmética de y
test_set = test_set.withColumn('y_mean', lit(m))
test_set.show(5)

+--------------------+--------+------------------+
|                   X|       y|            y_mean|
+--------------------+--------+------------------+
|[-61.935218703332...| 76100.0|206250.21533727524|
|[-61.925249251629...|106700.0|206250.21533727524|
|[-61.920264525778...| 78300.0|206250.21533727524|
|[-61.915279799926...| 66900.0|206250.21533727524|
|[-61.905310348223...| 69000.0|206250.21533727524|
+--------------------+--------+------------------+
only showing top 5 rows



In [0]:
re = RegressionEvaluator(labelCol='y', predictionCol='y_mean', metricName='mse')
print(f'mse: {re.evaluate(test_set)}')

mse: 13598344616.758198


## LinearRegression

In [0]:
lr = LinearRegression(featuresCol='X', labelCol='y', elasticNetParam=0.0)
lr_model = lr.fit(train_set)
y_pred = lr_model.transform(test_set)

re = RegressionEvaluator(labelCol='y', metricName='mse')
print(f'mse: {re.evaluate(y_pred)}')
y_pred.show(5)

mse: 6.134000912703967e-14
+--------------------+--------+------------------+------------------+
|                   X|       y|            y_mean|        prediction|
+--------------------+--------+------------------+------------------+
|[-61.935218703332...| 76100.0|206250.21533727524| 76099.99999942759|
|[-61.925249251629...|106700.0|206250.21533727524|106699.99999970393|
|[-61.920264525778...| 78300.0|206250.21533727524| 78300.00000000371|
|[-61.915279799926...| 66900.0|206250.21533727524| 66899.99999995674|
|[-61.905310348223...| 69000.0|206250.21533727524| 68999.99999969992|
+--------------------+--------+------------------+------------------+
only showing top 5 rows



## RandomForestRegressor

In [0]:
rf = RandomForestRegressor(featuresCol='X', labelCol='y')
rf_model = rf.fit(train_set)
y_pred = rf_model.transform(test_set)

re = RegressionEvaluator(labelCol='y', metricName='mse')
print(f'mse: {re.evaluate(y_pred)}')
y_pred.show(5)

mse: 525910458.1981659
+--------------------+--------+------------------+------------------+
|                   X|       y|            y_mean|        prediction|
+--------------------+--------+------------------+------------------+
|[-61.935218703332...| 76100.0|206250.21533727524|  92475.7915535873|
|[-61.925249251629...|106700.0|206250.21533727524|115720.22025346081|
|[-61.920264525778...| 78300.0|206250.21533727524| 94443.16872850516|
|[-61.915279799926...| 66900.0|206250.21533727524| 94028.43545639512|
|[-61.905310348223...| 69000.0|206250.21533727524| 99976.39598031531|
+--------------------+--------+------------------+------------------+
only showing top 5 rows



## GBTRegressor

In [0]:
gb = GBTRegressor(featuresCol='X', labelCol='y')
gb_model = gb.fit(train_set)
y_pred = gb_model.transform(test_set)

re = RegressionEvaluator(labelCol='y', metricName='mse')
print(f'mse: {re.evaluate(y_pred)}')
y_pred.show(5)

mse: 35084834.72792582
+--------------------+--------+------------------+-----------------+
|                   X|       y|            y_mean|       prediction|
+--------------------+--------+------------------+-----------------+
|[-61.935218703332...| 76100.0|206250.21533727524| 80457.3081295064|
|[-61.925249251629...|106700.0|206250.21533727524| 103833.769725325|
|[-61.920264525778...| 78300.0|206250.21533727524|80212.67675935487|
|[-61.915279799926...| 66900.0|206250.21533727524|69688.09642854854|
|[-61.905310348223...| 69000.0|206250.21533727524|  69693.235933113|
+--------------------+--------+------------------+-----------------+
only showing top 5 rows



### Hyperparameter tuning
Vou tentar melhorar um pouco o desempenho do modelo utilizando a técnica abaixo apenas para aprendizado e portfolio pois o desempenho da regressão linear é o suficiente.

In [0]:
gbt = GBTRegressor(featuresCol='X', labelCol='y')

paramGrid = ParamGridBuilder().addGrid(gbt.impurity, ['variance'])\
                              .addGrid(gbt.lossType, ['squared', 'absolute'])\
                              .addGrid(gbt.maxDepth, [5, 10, 20]).build()

crossval = CrossValidator(estimator=gbt,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol='y', metricName='mse'),
                          numFolds=2)

cvModel = crossval.fit(train_set)

## Best model
O melhos modelo, de longe, é o LinearRegression.